In [271]:
import requests
import pandas as pd
import json

def getAmount(base, quote, lpt_amount):
    urlBase='''https://www.okexcn.com/okexchain/v1/swap/liquidity/remove_quote/{base}_{quote}?liquidity={lptAmount}'''.format(base=base,quote=quote,lptAmount=lpt_amount)
    okbusdtr = requests.get(urlBase)
    okbusdt = json.dumps(okbusdtr.json())
    data = json.loads(okbusdt)
    df = pd.DataFrame(data['data'], columns=['denom','amount'])
    df['lqt_amount']=df['amount']
    
    return df

def getPrice(base):
    priceUrl='''https://www.okexcn.com/okexchain/v1/swap/liquidity/remove_quote/{base}_usdt-a2b?liquidity=0.01'''.format(base=base)
    priceR = requests.get(priceUrl)
    prices = json.dumps(priceR.json())
    price_data = json.loads(prices)
    baseAmount=price_data['data'][0]['amount']
    quoteAmount=price_data['data'][1]['amount']
    price=float(quoteAmount)/float(baseAmount)
    df = pd.DataFrame({'denom':base,'price':price},pd.Index(range(1)))
    return df

def getReword(poolname):
    rewordUrl='''https://www.okexcn.com/okexchain/v1/farm/earnings/{poolname}/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj'''.format(poolname=poolname)
    rewordR = requests.get(rewordUrl)
    reword = json.dumps(rewordR.json())
    reword_data = json.loads(reword)
    amount=reword_data['amount_yielded'][0]
    df = pd.DataFrame(amount, columns=['denom','amount'],index = [0])
    df['reword_amount']=df['amount']
    return df


In [280]:

#df1=getAmount("okb-c4d","usdt-a2b",0)
df2=getAmount("btck-ba9","okt",49.610546632517983358)
#df3=getAmount("ethk-c63","okt",0)


rr = requests.get('https://www.okexcn.com/okexchain/v1/accounts/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj')
rrData = json.loads(json.dumps(rr.json()))
rrdf = pd.DataFrame(rrData['data']['currencies'])
rrdf.rename(columns={'symbol':'denom','available':'amount'},inplace=True)
rrdf['account_amount']=rrdf['amount']
rrdf

#dfr1=getReword("btck_okt")
#dfr2=getReword("ethk_okt")
#dfr3=getReword("okb_usdt")

df = df2.append(rrdf)
df['amount']=df['amount'].astype("float64")
df['lqt_amount']=df['lqt_amount'].astype("float64")
df['account_amount']=df['account_amount'].astype("float64")
#df['reword_amount']=df['reword_amount'].astype("float64")
dfg=df.groupby('denom').sum()


okb=getPrice("okb-c4d")
btck=getPrice("btck-ba9")
ethk=getPrice("ethk-c63")
okt=getPrice("okt")
usdt = pd.DataFrame({'denom':'usdt-a2b','price':1},pd.Index(range(1)))

price = okb.append(btck).append(ethk).append(okt).append(usdt)
dfprice=pd.merge(dfg, price, on='denom')
dfprice['usdt'] = dfprice['amount']*dfprice['price']

dfprice['usdt'].sum()

144485.24506580736

In [281]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)
dfprice

,denom,amount,lqt_amount,account_amount,price,usdt
0,btck-ba9,0.46932912,0.46932912,0.00000001,52408.79783400,24596.97517594
1,okt,1671.54610947,343.52718602,1328.01892346,71.72298102,119888.26988986


In [162]:
dfprice['usdt'].sum()

104879.09652077063

In [270]:
dfr1

,denom,amount,reword_amount
0,okt,9.919790588199812444,9.919790588199812444


In [109]:
okbusdtr = requests.get('https://www.okexcn.com/okexchain/v1/swap/liquidity/remove_quote/okb-c4d_usdt-a2b?liquidity=0.99880969')

okbusdt = json.dumps(okbusdtr.json())
data = json.loads(okbusdt)
okbAmount=data['data'][0]['amount']
usdtAmount=data['data'][1]['amount']
price=float(usdtAmount)/float(okbAmount)
print ("OKB: ", okbAmount)
print ("USDT: ", usdtAmount)
print ("price: ", price)

OKB:  1489.404022518915920542
USDT:  13670.632226345126764748
price:  9.178592255461366


In [159]:
rr = requests.get('https://www.okexcn.com/okexchain/v1/accounts/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj')

rrData = json.loads(json.dumps(rr.json()))
rrdf = pd.DataFrame(rrData['data']['currencies'])
rrdf.rename(columns={'symbol':'denom','available':'amount'},inplace=True)
rrdf

,denom,amount,locked
0,ammswap_okb-c4d_usdt-a2b,0.000000002033978094,0
1,btck-ba9,0.000000005441568435,0
2,ethk-c63,0.000000000000000001,0
3,okb-c4d,0.000000008000152614,0
4,okt,267.075525711718378653,0
5,usdt-a2b,2275.764933267381108469,0
6,yfi-bb5,30000.000000000000000000,0


In [268]:
rrr = requests.get('https://www.okexcn.com/okexchain/v1/farm/earnings/btck_okt/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj')


In [269]:
rrrData = json.loads(json.dumps(rrr.json()))
rrrData


{'target_block_height': '790754',
 'amount_locked': {'denom': 'ammswap_btck-ba9_okt',
  'amount': '49.610546632517983358'},
 'amount_yielded': [{'denom': 'okt', 'amount': '17.471909215568899234'}]}

In [183]:
amount=rrrData['amount_yielded'][0]
df = pd.DataFrame(amount, columns=['denom','amount'],index = [0])
df

,denom,amount
0,okt,9.919790588199812444
